In [ ]:
from tensorflow.python.keras.applications.inception_v3 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.applications import InceptionV3
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization

num_clasees = 6

model = Sequential()
model.add(InceptionV3(include_top=False, 
                      pooling='avg', 
                      weights='../input/inceptionv3/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
                     ))
model.add(Dense(1024,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(6,activation='softmax'))

# model.add(Dense(num_clasees, activation="softmax"))

# do not need to train the pre train layer
model.layers[0].trainable=False

In [ ]:
# import tensorflow as tf
# tf.reset_default_graph()
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os, cv2, re, random
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
from keras import layers, models, optimizers
from keras import backend as K
from sklearn.model_selection import train_test_split
import os
print(os.listdir("../input/scene-classification/"))

TRAIN_DIR = '../input/scene-classification/train-scene classification/train/'
# TEST_DIR = '../input/test_WyRytb0.csv/'
train_images = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)]

y_test = pd.read_csv("../input/scene-classification/test_WyRytb0.csv")
y_train = pd.read_csv("../input/scene-classification/train-scene classification/train.csv")
y_train = y_train['label']
from keras.utils import to_categorical
y_train_categorical = to_categorical(y_train, 6)

train_images_list = []
test_images_list = []
a = list(y_test['image_name'])
for i in train_images:
    if str(i[63:]) not in a:
        train_images_list.append(i)
    if str(i[63:]) in a:
        test_images_list.append(i)
        
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]

train_images_list.sort(key=natural_keys)
test_images_list.sort(key=natural_keys)

def prepare_data(list_of_images):
    x = [] 
    for image in list_of_images:
        x.append(cv2.resize(cv2.imread(image), (150,150), interpolation=cv2.INTER_CUBIC))     
    return x
X_train = prepare_data(train_images_list)
X_test = prepare_data(test_images_list)

In [ ]:
X_train_prime, X_val, y_train_prime, y_val = train_test_split(X_train, y_train_categorical, random_state = 6, train_size = .80)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)


train_generator = train_datagen.flow(np.array(X_train_prime), y_train_prime, batch_size = 16, shuffle = True)
val_generator = val_datagen.flow(np.array(X_val), y_val, batch_size = 16, shuffle = True)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_history = model.fit_generator(
    train_generator,
    epochs = 60,
    steps_per_epoch=1000,
    validation_data=val_generator,
    validation_steps = 200)

In [ ]:
X_test = np.array(X_test)/255.0
prediction_probabilities = model.predict(X_test)
counter = range(1, len(test_images_list) + 1)
solution = pd.DataFrame({"id": counter, "label":list(prediction_probabilities)})
cols = ['label']
for i, j in enumerate(solution['label']):
    print(i+1)
    solution['label'][i] = np.argmax(j)
solution.to_csv("SceneClassification5.csv", index = False)